In [1]:
import numpy as np
import pandas as pd 
import astropy as ap
import matplotlib.pyplot as plt
from astropy.io import ascii

In [2]:
from scipy.stats import binned_statistic_dd
from scipy.stats import linregress
#from astroquery.mast import Catalogs
import mr_forecast as mr
pd.options.display.max_columns = 100
'''
note on mr_forecast: I replaced the Kipping & Chen function with the strictly analytical relationship implemented in Louie
et. al. 2018, which runs more quickly and for our purposes is sufficient. 

EXCEPT for planets with R > Rj 
''';

The goal of this notebook is to re-implement the matlab code written by Natalie in Python. The purpose is to select Tess planets for radial velocity follow-up with the Keck telescope in order to constrain their masses in preparation for further characterization with JWST. 

In [3]:
#Just getting the data - can use TESS or Barclay catalog 
#barclay catalog: https://arxiv.org/pdf/1804.05050.pdf
data_source = 'tess' #make this tess or barclay or exoplanet_archive
if data_source == 'tess':
    tess = pd.read_csv('toi_plus_list-2019-10-31.csv',delimiter=',',header=4)
    c = tess
if data_source == 'barclay':
    bc = ascii.read('barclay_2018_tess_catalog.txt')
    c = bc.to_pandas() #to generalize the use

In [4]:
#catalog_2 = star_df.dropna().merge(catalog,left_on='TICID',right_on='TIC ID').reset_index(drop=True)
#commented line above is when you need to do the MAST query
#star_info = pd.read_csv('TIC_wider_crossmatch.csv',delimiter=',',header=4)

#trying it now with a query to https://exofop.ipac.caltech.edu/tess/search.php for
#my TICID list, which returned fewer parameters but might be what I need 
if data_source == 'tess':
    star_info = pd.read_csv('TIC_star_info_oct_31_all.csv',delimiter=',',header=10)
    c['Stellar Mass'] = ((10**c['Surface Gravity Value']) * ((c['Star Radius Value']*6.9551e10)**2) 
                         / 6.67e-8 ) / 1.99e33 
    catalog_2 = pd.merge(c,star_info, left_on = 'TIC', right_on = 'Target')
    c2 = catalog_2.drop(columns=['TFOP SG1a','TFOP SG1b','TFOP SG2',\
                                'TFOP SG3','TFOP SG4','TFOP SG5','TFOP Master',\
                                'TOI Disposition'])
    c = c2.sort_values('Full TOI ID')

In [5]:
#naming all the variables, so it's easier to switch from bc to tess
#future note to self: better to just do all the below as functions, rather than hard-coding all the variables
#at the top; make this change when you've got some spare time

#note: might be easier to have a dictionary of keys, which I could then make calls to 
#by having functions accept the dictionary as input, rather than each key individually...
#not that it makes that much of a difference but it might clean things up a bit

if data_source == 'barclay':
    #rp = catalog['Planet-radius']
    #ms = catalog['Star-mass']
    #pp = catalog['Planet-period']
    #Ts = catalog['Star-teff']
    #ars = catalog['Ars'] #this is the semimajor axis of the orbit over the stellar radius
    #Jmag = catalog['Jmag']
    #Vmag = catalog['Vmag']
    #rs = catalog['Star-radius']
    #dec = catalog['DEdeg'] #declination
    #Fp = catalog['Insol']

    rp_key = 'Planet-radius'
    ms_key = 'Star-mass'
    pp_key = 'Planet-period'
    Ts_key = 'Star-teff'
    ars_key = 'Ars'
    Jmag_key = 'Jmag'
    Vmag_key = 'Vmag'
    rs_key = 'Star-radius'
    dec_key = 'DEdeg'
    Fp_key = 'Insol'
    
if data_source == 'tess':
    #rp = catalog['Planet Radius (R_Earth)']
    #ms = catalog['st_mass']  
    #pp = catalog['Period (days)']
    #Ts = catalog['Stellar Teff (K)']
    #ars = catalog['Ars'] #this is the semimajor axis of the orbit over the stellar radius
    #Jmag = catalog['Jmag']
    #Vmag = catalog['Vmag']
    #rs = catalog['Stellar Radius (R_Sun)']
    #dec = catalog['Dec (degrees)'] #declination
    #Fp = catalog['Planet Insolation (Earth flux)']
    
    rp_key = 'Planet Radius Value'
    ms_key = 'Stellar Mass' 
    pp_key = 'Orbital Period Value'
    Ts_key = 'Effective Temperature Value'
    ars_key = 'Ars'
    Jmag_key = 'J mag' #spaces in mags comes from the exofop catalog; remove space if getting info from MAST
    Vmag_key = 'V mag'
    rs_key = 'Star Radius Value' 
    dec_key = 'TIC Declination'
    Fp_key = 'Effective Stellar Flux Value'
    mp_key = 'pl_masses'
    mp_units = 'Earth'
    
    #below are the keys for exofop; above are from https://tev.mit.edu:8010/data/collection/193/ 
        #which I think is standard from here on out
    #rp_key = 'Planet Radius (R_Earth)'
    #ms_key = 'Stellar Mass' #the from MAST and a god awful name is 'mass'; change this! 
    #                                #it's "Stellar Mass (M_Sun)" from exofop which is much better
    #pp_key = 'Period (days)'
    #Ts_key = 'Stellar Eff Temp (K)'
    #ars_key = 'Ars'
    #Jmag_key = 'J mag' #spaces in mags comes from the exofop catalog; remove space if getting info from MAST
    #Vmag_key = 'V mag'
    #rs_key = 'Stellar Radius (R_Sun)' 
    #dec_key = 'Dec_y'
    #Fp_key = 'Planet Insolation (Earth Flux)'
    #mp_key = 'pl_masses'
    #mp_units = 'Earth'
    
if data_source == 'exoplanet_archive':
    
    rp_key = 'pl_radj' #FINISH FILLING THIS IN BELOW HERE 
    ms_key = 'st_mass'
    pp_key = 'pl_orbper'
    Ts_key = 'st_teff'
    ars_key = 'Ars'
    Jmag_key = 'st_j'
    Vmag_key = 'st_vj'
    rs_key = 'st_rad'
    dec_key = 'dec'
    Fp_key = 'pl_insol'

In [6]:
exoarch_keys = ['pl_rade','st_mass','pl_orbper','st_teff','Ars','st_j',
                'st_vj','st_rad','dec','pl_insol', 'pl_masse']
all_planets = pd.read_csv('transits_above_v14.csv',header=46)
ap = all_planets
known_planets = pd.DataFrame({
    rp_key : ap[exoarch_keys[0]],
    ms_key : ap[exoarch_keys[1]],
    pp_key : ap[exoarch_keys[2]],
    rs_key : ap[exoarch_keys[7]],
    Ts_key : ap[exoarch_keys[3]],
    Jmag_key : ap[exoarch_keys[5]],
    Vmag_key : ap[exoarch_keys[6]], 
    dec_key : ap[exoarch_keys[8]],
    Fp_key : ap[exoarch_keys[9]]
})

In [7]:
known_planets.columns

Index(['Planet Radius Value', 'Stellar Mass', 'Orbital Period Value',
       'Star Radius Value', 'Effective Temperature Value', 'J mag', 'V mag',
       'TIC Declination', 'Effective Stellar Flux Value'],
      dtype='object')

In [8]:
def mass_getter(planet_data, rad_key, mass_key):
    planet_mass = np.zeros(len(planet_data[rad_key]))
    big_ones = planet_data[rad_key]>14.26 #i.e. too big for the empirical relation
    masked_ones = pd.isna(planet_data[mass_key])
    
    i = 0
    while i < len(planet_mass):
        if masked_ones[i] and not big_ones[i]:
            #if the entry is masked (i.e. zero) AND it's small, use the empirical relationship
            planet_mass[i] = (0.9718 * ((planet_data[rad_key][i])**3.58)) if planet_data[rad_key][i] < 1.23 \
            else (1.436 * ((planet_data[rad_key][i])**1.7)) #Earth Masses
        i+=1
        
    #planet_mass[np.logical_and(big_ones, masked_ones)] = \
    #    mr.Rpost2M(planet_data[rad_key][np.logical_and(big_ones, masked_ones)],unit='Earth')
    #for the entries which are masked and big, apply the statistical relationship
    
    planet_mass[np.logical_and(big_ones, masked_ones)] = 317.8 #anything really big is just Jupiter fck it lol
                                                                #it's outside our bin regime anyway
    
    planet_mass[~masked_ones] = planet_data[mass_key][~masked_ones]
    #for unmasked entries, take the existing values
    
    return np.round(planet_mass,4)

known_planets[mp_key] = mass_getter(ap,exoarch_keys[0],exoarch_keys[10])

In [49]:
def ars_from_t(period,star_mass,star_radius):
    '''
    period should be given in days; star mass and radius in
    terms of their value in solar units. These units seem
    to be pretty much standard across exoplanet databases
    so I have not implemented any sort of unit options
    in this function.
    '''
    G = 6.6726e-11
    Msun = 1.989e30
    Mstar = star_mass * Msun
    period_s = period * 86400
    a_m = (G*Mstar*(period_s**2) / (4 * (np.pi**2))) ** (1/3)
    star_radius_m = star_radius * 6.9551e8 #sun radius in m
    ars = a_m / star_radius_m
    #this value is correct for earth 
    return ars

known_planets[ars_key] = ars_from_t(known_planets[pp_key],known_planets[ms_key],known_planets[rs_key])

In [50]:
if data_source == 'tess':
    c = c[np.logical_and.reduce((c[rp_key]>0, c[pp_key]>0, c['Source Pipeline']=='spoc'))]
    c[ars_key] = ars_from_t(c[pp_key], c[ms_key], c[rs_key]) #because there is no ars data in the tess datafile
    c = c.reset_index(drop=True)

In [51]:
#Calculating Optimistic mass based on relation in Chen & Kipping 2016 (https://arxiv.org/pdf/1603.08614.pdf) and direct 
#empircal implementation by Louie et. al. 2018 (https://iopscience.iop.org/article/10.1088/1538-3873/aaa87b)
def m_from_r(radii):
    masses = np.zeros(len(radii))
    i = 0
    while i < len(radii):
        masses[i] = (0.9718 * (radii[i]**3.58)) if radii[i] < 1.23\
        else (1.436 * (radii[i]**1.7)) if radii[i] < 14.26\
        else -1
        #don't use this analysis for planets outside the neptune regime: return -1
        i += 1
    return masses

c[mp_key] = m_from_r(c[rp_key])

In [52]:
def k_amp_finder(star_mass,star_radius,planet_mass,ars,mp_units):
    '''
    finds the amplitude of stellar oscillations due to the planet's orbit 
    star_mass: units of m_sun
    star_radius: units of r_sun
    planet_mass: units of m_earth or m_jup (should be able to set "mp_units = mp_units" as that 
                                            variable is named higher up in the chain. Note "Earth"
                                            is capitalized, if you set this manually. 
    ars: ratio of semimajor axis and stellar radius. This is used as an input instead of a alone 
                        because a is often not directly available from these datasets. 
    '''
    
    G = 6.67e-11 #SI units
    Msun = 1.98e30 #kg
    Rsun = 6.9551e8 #meters
    v_star = np.sqrt(G*star_mass*Msun/(ars*star_radius*Rsun)) 
    #this is actually backwards
    
    mp_factor = 5.972e24 if mp_units == 'Earth' else 1.898e27
    v_pl = v_star * ((planet_mass*mp_factor)/(star_mass*Msun))
    
    return v_pl
    
c['K_amp'] = k_amp_finder(c[ms_key],c[rs_key],c[mp_key],c[ars_key],mp_units)

In [53]:
#mjupiter=1.898e27; #kg
#mearth=5.972e24; #kg
#
#kamp = 203*(T(:,22).^(-1/3)).*(plmass*mearth/mjupiter)./((T(:,16) + 9.548e-4*plmass*mearth/mjupiter).^(2/3));

def k_amp_natalie(period,planet_mass,star_mass):
    '''
    from https://exoplanetarchive.ipac.caltech.edu/docs/poet_calculations.html
    '''
    mjupiter=1.898e27 #kg
    mearth=5.972e24 #kg
    k_amp = 203 * (period**(-1/3)) * (planet_mass*mearth/mjupiter)\
                            / (((star_mass) + (9.548e-4 * planet_mass * mearth/mjupiter) ) ** (2/3))
    return k_amp

c['K_amp_n'] = k_amp_natalie(c[pp_key],c[mp_key],c[ms_key])

In [54]:
c[['K_amp','K_amp_n']][np.logical_or(np.logical_and(c['K_amp_n'] < 2, c['K_amp'] > 2), \
                                    np.logical_and(c['K_amp'] < 2, c['K_amp_n'] > 2))]
#with the updated data, our values DO differ for TOI 262.01

,K_amp,K_amp_n
228,2.004816,1.998977


In [55]:
#Calculating Webb SNR Proxy
#taken from Kempton et. al. : https://arxiv.org/pdf/1805.03671.pdf
#note: implement "ars" and "mp" being "None" so there's an option for the
#function to calculate them internally, making it more generally applicable

def get_TSM(planet_radius,star_radius,star_teff,Jmag,planet_mass,ars):
    
    scale_factors = np.zeros(len(planet_radius))
    i = 0
    while i < len(planet_radius):
        #from Table 1 in Kempton et. al. 
        scale_factors[i] = 0.19 if planet_radius[i]<1.5\
        else 1.26 if np.logical_and(planet_radius[i]>1.5,planet_radius[i]<2.75)\
        else 1.28 if np.logical_and(planet_radius[i]>2.75, planet_radius[i]<4)\
        else 1.15
        i += 1
    
    Teq = star_teff * (np.sqrt(1/ars)*(0.25**0.25)) #eqn 3 in Kempton et. al <- this is how I calculated TSM
   
    #TSM is transmission spectroscopy metric
    TSM = scale_factors * (planet_radius**3) * Teq * (10**(-Jmag/5)) / (planet_mass * (star_radius**2))
    return TSM

c['TSM'] = get_TSM(c[rp_key],c[rs_key],c[Ts_key],c[Jmag_key],c[mp_key],c[ars_key])
known_planets['TSM'] = get_TSM(known_planets[rp_key],known_planets[rs_key],
                                        known_planets[Ts_key],known_planets[Jmag_key],
                                        known_planets[mp_key],known_planets[ars_key])

In [56]:
#Calculating Webb SNR Proxy
#taken from Kempton et. al. : https://arxiv.org/pdf/1805.03671.pdf
#note: implement "ars" and "mp" being "None" so there's an option for the
#function to calculate them internally, making it more generally applicable

def get_natalie_TSM(planet_radius,star_radius,star_teff,Jmag,planet_mass,insol_flux):
    
    scale_factors = np.zeros(len(planet_radius))
    i = 0
    while i < len(planet_radius):
        #from Table 1 in Kempton et. al. 
        scale_factors[i] = 0.19 if planet_radius[i]<1.5\
        else 1.26 if np.logical_and(planet_radius[i]>1.5,planet_radius[i]<2.75)\
        else 1.28 if np.logical_and(planet_radius[i]>2.75, planet_radius[i]<4)\
        else 1.15
        i += 1
    
    rsun=696265; #km
    au=149598000; #km
    albedo=0; #.3; %3; %0.1; Bond albedo
    rearth=6371; #km
    tsun=5777;
    Teq = (((insol_flux)*(1/4))**0.25)*np.sqrt(rsun/au)*tsun 

    #TSM is transmission spectroscopy metric
    TSM = scale_factors * (planet_radius**3) * Teq * (10**(-Jmag/5)) / (planet_mass * (star_radius**2))
    return TSM

c['nTSM'] = get_natalie_TSM(c[rp_key],c[rs_key],c[Ts_key],c[Jmag_key],c[mp_key],c[Fp_key])
known_planets['nTSM'] = get_natalie_TSM(known_planets[rp_key],known_planets[rs_key],
                                        known_planets[Ts_key],known_planets[Jmag_key],
                                        known_planets[mp_key],known_planets[Fp_key])

In [18]:
#desirables = np.logical_and.reduce((Jmag<12, Vmag<14, dec<20, catalog['K_amp']>2))
desirables = np.logical_and(c[dec_key]>-20, c['K_amp']>2)

#NOTE: maybe ought to change the dec key back to the normal one - the _x comes, again, from MAST

#jmag is webb limit, Kmag is keck limit, dec is to simulate fraction of catalog visible from keck, vs is k_amp for star

In [90]:
catalog_cleaned = c[desirables]#.drop_duplicates(subset='TOI') #removed for barclay catalog
cc = catalog_cleaned.reset_index(drop=True)

In [20]:
#cc.to_csv(path_or_buf=r'tess_data_with_ticid_info.csv',index=False);
#done so that I can just reload here instead of going through that 
#god awful loading of all the TICID info from MAST again. 

In [64]:
#cc = pd.read_csv('tess_data_with_ticid_info.csv')
id_key = 'Full TOI ID'
kp = known_planets
kp[id_key] = ap['pl_name']
cc = cc.append(kp, sort=False)

In [65]:
#defining new bins - as per Natalie
rad_bins = 10**(np.linspace(0,1,6))
fpl_bins = 10**(np.linspace(-1,4,6))
tef_bins = np.array([2500,3900,5200,6500])
all_bins = [rad_bins, fpl_bins, tef_bins]

def binning_function(dataset,bins,id_key,sort_val):
    
    rad_bins = bins[0]
    fpl_bins = bins[1]
    tef_bins = bins[2]

    pre_bin = dataset.assign(
        radius_bin = pd.cut(dataset[rp_key],bins=rad_bins,labels = [1,2,3,4,5]),
        insol_bin = pd.cut(dataset[Fp_key],bins=fpl_bins,labels = [1,2,3,4,5]),
        st_Teff_bin = pd.cut(dataset[Ts_key],bins=tef_bins,labels = [1,2,3])
    )
        #pd.cut returns the bin number (or label - ints chosen here for ease)
        #of each row based on its place within a specified column. 
    
    binned = pre_bin.dropna(subset=['radius_bin','insol_bin','st_Teff_bin']).groupby(['radius_bin',\
                                    'insol_bin','st_Teff_bin']).apply(lambda _pre_bin:\
                                    _pre_bin.sort_values(by=[sort_val],ascending=False)).reset_index(level = 3,drop=True)
                    #this multi-line call: 
                        #1) drops values which are not in any of the desired bins 
                        #2) groups within those bins
                        #3) sorts by TSM (the lambda thing is necessary because "groupby" produces a "groupby object"
                                #which can't be operated on normally)
                        #4) drops all indexes which are not the bin numbers, which were just 1 to N anyway and therefore
                                #were worthless 
    
    all_idx = binned.index.to_list()
    unique_idx = []
    for element in all_idx: 
        if element not in unique_idx:
            unique_idx.append(element)
    
    binned['priority'] = np.zeros(len(binned))
    for idx in unique_idx:
        
        bin_items = len(binned.loc[idx].sort_values(sort_val,ascending=False).iloc[0:3].sort_values(Vmag_key)[id_key])
            #the number of objects in each bin
            
        if bin_items >= 3:
            binned.loc[binned[id_key] == binned.loc[idx].sort_values(sort_val,ascending=False).iloc[0:3]\
                       .sort_values(Vmag_key)[id_key].iloc[0],'priority'] = 1
            binned.loc[binned[id_key] == binned.loc[idx].sort_values(sort_val,ascending=False).iloc[0:3]\
                       .sort_values(Vmag_key)[id_key].iloc[1],'priority'] = 2
            binned.loc[binned[id_key] == binned.loc[idx].sort_values(sort_val,ascending=False).iloc[0:3]\
                       .sort_values(Vmag_key)[id_key].iloc[2],'priority'] = 3
            continue
            
        elif bin_items == 2:
            binned.loc[binned[id_key] == binned.loc[idx].sort_values(sort_val,ascending=False).iloc[0:3]\
                       .sort_values(Vmag_key)[id_key].iloc[0],'priority'] = 1
            binned.loc[binned[id_key] == binned.loc[idx].sort_values(sort_val,ascending=False).iloc[0:3]\
                       .sort_values(Vmag_key)[id_key].iloc[1],'priority'] = 2
            continue
            
        elif bin_items == 1:
            binned.loc[binned[id_key] == binned.loc[idx].sort_values(sort_val,ascending=False).iloc[0:3]\
                       .sort_values(Vmag_key)[id_key].iloc[0],'priority'] = 1
    
        #this is a HIDEOUS call but the idea is: 
            #you are going into each bin sequentially (by index), sorting by TSM, then sorting those top 3 by Vmag. 
            #then, you are taking out the TOI value of the top entry there (i.e., highest priority)
            #THEN, you are indexing that TOI in the list, .loc'ing to that row and the priority column, and setting 
            #THAT entry to 1. Then repeating this for the other priority values 
            
            #all these if statements are a lot but unless I want to predefine 
            #how many are in each bin (?) I think this is the fastest way to go, and as long as 
            #TESS keeps its number of targets < 10^5 or something this shouldn't be unacceptably
            #long in terms of its run time 
    return binned

In [91]:
binned = binning_function(cc,all_bins,'Full TOI ID','TSM')

In [92]:
#rewriting this to include ExoArch planets
my_tois_or = binned[binned['priority']==1].reset_index(drop=True)[id_key].values
my_tois_or

array([ 455.01,  732.01,  635.01,  732.02,  663.01, 1235.01, 1285.01,
        238.01,  266.01, 1242.01,  638.01,  233.01,  620.01,  560.01,
       1339.01, 1246.02, 1136.02,  526.01,  538.01,  532.01, 1136.01,
        465.01, 1258.01,  680.01, 1278.01,  507.01, 1268.01,  421.01,
        460.01, 1264.01, 1135.01,  390.01])

In [89]:
my_tois = binned[binned['priority']==1].reset_index(drop=True)[id_key].values
my_tois

array([455.01, 'GJ 357 b', 'K2-233 c', 'K2-65 b', 'K2-233 b', 'K2-138 b',
       635.01, 732.02, 'K2-21 c', 663.01, 'Wolf 503 b', 'K2-74 b',
       1285.01, 238.01, 'pi Men c', 1242.01, 638.01, 233.01, 'K2-266 e',
       620.01, 560.01, 1339.01, 1246.02, 1136.02, 526.01, 538.01, 532.01,
       1136.01, 465.01, 1258.01, 680.01, 1278.01, 507.01, 1268.01, 421.01,
       460.01, 1264.01, 1135.01, 390.01], dtype=object)

In [93]:
counter = 0
cut_tois = []
for old_toi in my_tois_or:
    if old_toi not in my_tois:
        print(old_toi)
        cut_tois.append(old_toi)

732.01
1235.01
266.01


From the most recent sector:

TOI       Rp          Teq             J

1235.01   ~2.1        703             8.7

1242.01   ~1.8        1670            10.5

1246.02   ~2.8        955             10.3

1258.01   ~5.0        1059            8.6

1264.01   ~7.6        1154            9.6

1268.01   ~8.9        784             9.4

1278.01   ~9.3        408             10.6

1285.01   ~1.9        841             9.5

In [62]:
binned[binned['priority']==1][[id_key,rp_key,'priority','K mag']]

Full TOI ID  Planet Radius Value  priority  \
radius_bin insol_bin st_Teff_bin                                              
1          2         1                455.01             1.368307       1.0   
           3         1              GJ 357 b             1.217000       1.0   
                     1              GJ 357 b             1.217000       1.0   
                     2              K2-233 c             1.335000       1.0   
                     2              K2-233 c             1.335000       1.0   
           4         2              K2-233 b             1.398000       1.0   
                     2              K2-233 b             1.398000       1.0   
                     3              K2-138 b             1.570000       1.0   
                     3              K2-138 b             1.570000       1.0   
           5         3                635.01             1.503946       1.0   
2          2         1                732.02             1.785398       1.0   
                     2               K2-21 c             2.490000       1.0   
                     2               K2-21 c             2.490000       1.0   
           3         1                663.01             2.224192       1.0   
                     2            Wolf 503 b             2.030000       1.0   
                     2            Wolf 503 b             2.030000       1.0   
                     3               K2-74 b             2.310000       1.0   
                     3               K2-74 b             2.310000       1.0   
           4         1               1285.01             1.866263       1.0   
                     2                238.01             1.676542       1.0   
                     3              pi Men c             2.042000       1.0   
                     3              pi Men c             2.042000       1.0   
           5         2               1242.01             1.817591       1.0   
                     3                638.01             1.940277       1.0   
3          2         1                233.01             2.570156       1.0   
           3         1                620.01             2.729669       1.0   
                     2                560.01             2.847985       1.0   
                     3               1339.01             3.271100       1.0   
           4         2               1246.02             2.845102       1.0   
                     3               1136.02             3.002270       1.0   
4          2         1                526.01             6.214510       1.0   
           3         1                538.01             5.984065       1.0   
                     2                532.01             5.611109       1.0   
                     3               1136.01             4.798384       1.0   
           4         2                465.01             5.635860       1.0   
                     3               1258.01             4.988378       1.0   
           5         3                680.01             5.741712       1.0   
5          2         1               1278.01             9.281245       1.0   
           3         1                507.01             9.812343       1.0   
                     2               1268.01             8.935117       1.0   
                     3                421.01             6.445067       1.0   
           4         1                460.01             6.597837       1.0   
                     2               1264.01             7.621986       1.0   
                     3               1135.01             9.001699       1.0   
           5         3                390.01             8.656736       1.0   

                                      K mag  
radius_bin insol_bin st_Teff_bin             
1          2         1             6.496000  
           3         1                  NaN  
                     1                  NaN  
                     2                  NaN  
                     2                  NaN

In [43]:
c[np.round(c[id_key], 0) == 1339][[id_key,dec_key,pp_key,rp_key,Fp_key,'Planet Equilibrium Temperature (K) Value',
                                   Ts_key,Vmag_key,Jmag_key,'pl_masses','TSM','Public Comment']]
#so my k_amp and Natalie's are BOTH below 2! What the heck?? 
#okay, I suspect the rows error crept into Natalie's code somehow, because both of
#our formulae return K_amp = 1.72 for TOI 455.01. Either that, or there was a 
#typo when she copied the list into an email for me. 

,Full TOI ID,TIC Declination,Orbital Period Value,Planet Radius Value,Effective Stellar Flux Value,Planet Equilibrium Temperature (K) Value,Effective Temperature Value,V mag,J mag,pl_masses,TSM,Public Comment
634,1339.01,66.850576,8.879230,3.271100,39.310145,638.000000,5555.7,8.97,7.597,10.767972,132.785168,possible multi; some transits fell in gaps
635,1339.02,66.850576,11.062812,3.009164,73.910839,747.819495,5555.7,8.97,7.597,9.343560,110.712623,part of possible multi-planet system


In [57]:
#np.savetxt(fname='nicholas_target_list.txt',X=vecstr(my_tois),fmt='%s')
binned[np.round(binned[id_key], 0) == 1339]

Source Pipeline        TIC  Full TOI ID  \
radius_bin insol_bin st_Teff_bin                                           
3          3         3                      spoc  269701147      1339.02   
                     3                      spoc  269701147      1339.01   

                                  TIC Right Ascension  TIC Declination  \
radius_bin insol_bin st_Teff_bin                                         
3          3         3                     302.023979        66.850576   
                     3                     302.023979        66.850576   

                                  TMag Value  TMag Uncertainty  \
radius_bin insol_bin st_Teff_bin                                 
3          3         3                 8.293             0.006   
                     3                 8.293             0.006   

                                  Orbital Epoch Value  \
radius_bin insol_bin st_Teff_bin                        
3          3         3                    1743.556395   
                     3                    1741.996126   

                                  Orbital Epoch Uncertainty  \
radius_bin insol_bin st_Teff_bin                              
3          3         3                             0.001723   
                     3                             0.002530   

                                  Orbital Period Value  \
radius_bin insol_bin st_Teff_bin                         
3          3         3                       11.062812   
                     3                        8.879230   

                                  Orbital Period Uncertainty  \
radius_bin insol_bin st_Teff_bin                               
3          3         3                              0.002426   
                     3                              0.000820   

                                  Transit Duration (hours) Value  \
radius_bin insol_bin st_Teff_bin                                   
3          3         3                                  5.256185   
                     3                                  3.007854   

                                  Transit Duration (hours) Uncertainty  \
radius_bin insol_bin st_Teff_bin                                         
3          3         3                                        0.250002   
                     3                                        0.156000   

                                  Transit Depth Value  \
radius_bin insol_bin st_Teff_bin                        
3          3         3                    1063.671990   
                     3                    1257.358469   

                                  Transit Depth Uncertainty Sectors  \
radius_bin insol_bin st_Teff_bin                                      
3          3         3                            45.211372      16   
                     3                             0.576369      16   

                                                              Public Comment  \
radius_bin insol_bin st_Teff_bin                                               
3          3         3                  part of possible multi-planet system   
                     3            possible multi; some transits fell in gaps   

                                  Surface Gravity Value  \
radius_bin insol_bin st_Teff_bin                          
3          3         3                             4.49   
                     3                             4.49   

                                  Surface Gravity Uncertainty  Signal ID  \
radius_bin insol_bin st_Teff_bin                                           
3          3         3                                   0.08          2   
                     3                                   0.08          1   

                                  Star Radius Value  Star Radius Uncertainty  \
radius_bin insol_bin st_Teff_bin                                               
3          3         3                     0.920435                 0.046312   
       

In [55]:
print(binned[binned['priority']!=0].sort_values([id_key])[id_key].values)

[ 233.01  238.01  256.02  266.01  278.01  390.01  421.01  455.01  460.01
  461.01  465.01  469.01  482.01  485.01  506.01  507.01  516.01  519.01
  521.01  526.01  529.01  532.01  538.01  543.01  544.01  560.01  561.02
  564.01  620.01  634.01  635.01  638.01  654.01  663.01  663.02  669.01
  680.01  732.01  732.02  736.01  736.02 1135.01 1136.01 1136.02 1194.01
 1201.01 1235.01 1239.01 1242.01 1244.01 1246.02 1251.01 1254.01 1256.01
 1258.01 1260.01 1264.01 1268.01 1272.01 1274.01 1275.01 1276.01 1278.01
 1284.01 1285.01 1288.01 1289.01 1339.01 1339.02 1340.01]


In [25]:
natalie_tois = np.array([123.01,129.01,134.01,140.01,141.01,177.01,179.01,200.01,216.01,221.01,245.01,270.02,282.01,431.02,455.01,482.01,507.01,509.01,510.01,523.01,526.01,533.01,539.01,554.01,560.01,561.02,576.01,620.01,672.01,682.01,712.02,732.01,783.01,784.01,788.01,793.01,821.01,836.02,865.03])
'''
natalie_data = pd.read_csv('targetList.csv',names=['TICID','RA','DEC','Vmag','Tmag','Jmag',
                                                   'st_teff','st_rad','period','pl_rade',
                                                   'Insol','teq','kamp','TSM'])
'''
#natalie_data
##binned2 = binning_function(cc,all_bins,'TICID')
#natalie_tics = binned2[binned2['priority']==1].reset_index(drop=True).sort_values(id_key)[id_key].values
natalie_tois

array([123.01, 129.01, 134.01, 140.01, 141.01, 177.01, 179.01, 200.01,
       216.01, 221.01, 245.01, 270.02, 282.01, 431.02, 455.01, 482.01,
       507.01, 509.01, 510.01, 523.01, 526.01, 533.01, 539.01, 554.01,
       560.01, 561.02, 576.01, 620.01, 672.01, 682.01, 712.02, 732.01,
       783.01, 784.01, 788.01, 793.01, 821.01, 836.02, 865.03])

In [26]:
binned[np.round(binned[id_key],0) == 732]

,,,Source Pipeline,TIC,Full TOI ID,TIC Right Ascension,TIC Declination,TMag Value,TMag Uncertainty,Orbital Epoch Value,Orbital Epoch Uncertainty,Orbital Period Value,Orbital Period Uncertainty,Transit Duration (hours) Value,Transit Duration (hours) Uncertainty,Transit Depth Value,Transit Depth Uncertainty,Sectors,Public Comment,Surface Gravity Value,Surface Gravity Uncertainty,Signal ID,Star Radius Value,Star Radius Uncertainty,Planet Radius Value,Planet Radius Uncertainty,Planet Equilibrium Temperature (K) Value,Effective Temperature Value,Effective Temperature Uncertainty,Effective Stellar Flux Value,Signal-to-noise,Centroid Offset,Alerted,Edited,Stellar Mass,Target,RA,Dec,V mag,J mag,H mag,K mag,Ars,pl_masses,K_amp,K_amp_n,TSM,nTSM,radius_bin,insol_bin,st_Teff_bin,priority
radius_bin,insol_bin,st_Teff_bin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3,1,spoc,36724087,732.01,154.6464,-11.7167,10.534,0.058,1543.911480,0.001189,0.768418,0.000067,0.816472,0.266768,1252.849899,105.63316,9,NaN,4.94957,0.423263,1,0.3,0.054,1.134371,0.573323,770.585234,3329.0,72.0,83.330451,11.443317,False,2019-06-06T16:40:58+00:00,2019-10-17 00:41:48.757745+00:00,0.292039,36724087,154.644903,-11.717801,13.14,9.007,8.439,8.204001,7.810036,1.526160,2.424886,2.417812,26.868814,26.858943,1,3,1,1.0
2,2,1,spoc,36724087,732.02,154.6464,-11.7167,10.534,0.058,1546.848329,0.001409,12.254218,0.002019,1.361689,0.332050,3416.662447,283.46980,9,NaN,4.94957,0.423263,2,0.3,0.054,1.785398,0.730371,306.143276,3329.0,72.0,2.075964,11.924323,False,2019-06-06T16:40:58+00:00,2019-10-17 00:41:48.764789+00:00,0.292039,36724087,154.644903,-11.717801,13.14,9.007,8.439,8.204001,49.481755,3.846839,2.428282,2.421160,109.497802,109.457586,2,2,1,1.0


In [50]:
binned[binned['Planet Equilibrium Temperature (K) Value'] < 373]

Source Pipeline        TIC  Full TOI ID  \
radius_bin insol_bin st_Teff_bin                                           
2          2         1                      spoc  181804752       736.01   
                     1                      spoc   43064903      1276.01   
                     1                      spoc   36724087       732.02   
                     1                      spoc  429358906       782.01   

                                  TIC Right Ascension  TIC Declination  \
radius_bin insol_bin st_Teff_bin                                         
2          2         1                     165.691500       -16.406200   
                     1                     302.667462        40.668998   
                     1                     154.646400       -11.716700   
                     1                     183.921100       -18.910200   

                                  TMag Value  TMag Uncertainty  \
radius_bin insol_bin st_Teff_bin                                 
2          2         1               13.5080          0.059000   
                     1               12.5405          0.007423   
                     1               10.5340          0.058000   
                     1               12.1910          0.040000   

                                  Orbital Epoch Value  \
radius_bin insol_bin st_Teff_bin                        
2          2         1                    1546.508996   
                     1                    1712.548761   
                     1                    1546.848329   
                     1                    1577.042852   

                                  Orbital Epoch Uncertainty  \
radius_bin insol_bin st_Teff_bin                              
2          2         1                             0.001156   
                     1                             0.004739   
                     1                             0.001409   
                     1                             0.002817   

                                  Orbital Period Value  \
radius_bin insol_bin st_Teff_bin                         
2          2         1                        4.990070   
                     1                        6.264298   
                     1                       12.254218   
                     1                       16.047204   

                                  Orbital Period Uncertainty  \
radius_bin insol_bin st_Teff_bin                               
2          2         1                              0.000444   
                     1                              0.002524   
                     1                              0.002019   
                     1                              0.004003   

                                  Transit Duration (hours) Value  \
radius_bin insol_bin st_Teff_bin                                   
2          2         1                                  1.150579   
                     1                                  2.389982   
                     1                                  1.361689   
                     1                                  1.709139   

                                  Transit Duration (hours) Uncertainty  \
radius_bin insol_bin st_Teff_bin                                         
2          2         1                                        0.162682   
                     1                                        0.306185   
                     1                                        0.332050   
                     1                                        0.538583   

                                  Transit Depth Value  \
radius_bin insol_bin st_Teff_bin                        
2          2         1                   17848.071663   
                     1                    9097.780143   
                     1                    3416.662447   
                     1                    4333.162901   

                                  Transit Depth Uncertainty Sectors  \
radius_bin

In [70]:
mismatch_tois = []
counter = 0
for toi in natalie_tois:
    if toi in my_tois:
        counter += 1
    if toi not in my_tois:
        print(toi)
        mismatch_tois.append(toi)
print(counter)
#Many TICIDs of missing TOIs ARE in my star_data list (e.g. 154618248) 
#so I'm not sure how they got excluded from the other list
# - maybe K_amp differences? Investigate tomorrow

#okay, so, 140.01 HAS all the values, but in its bin, it's beaten by 408, 670, and 467, 
#MISSING: 455.01 (k_amp), 523.01 (no stellar mass)

140.01
455.01
523.01
533.01
561.02
620.01
672.01
682.01
31


In [56]:
pd.options.display.max_columns = 100
binned.loc[4,4,2]

Source Pipeline        TIC  Full TOI ID  \
radius_bin insol_bin st_Teff_bin                                           
4          4         2                      spoc  270380593       465.01   
                     2                      spoc  417948359      1272.01   

                                  TIC Right Ascension  TIC Declination  \
radius_bin insol_bin st_Teff_bin                                         
4          4         2                      32.781778         2.418047   
                     2                     199.196626        49.861039   

                                  TMag Value  TMag Uncertainty  \
radius_bin insol_bin st_Teff_bin                                 
4          4         2                10.661             0.018   
                     2                11.024             0.006   

                                  Orbital Epoch Value  \
radius_bin insol_bin st_Teff_bin                        
4          4         2                    1414.135825   
                     2                    1713.031745   

                                  Orbital Epoch Uncertainty  \
radius_bin insol_bin st_Teff_bin                              
4          4         2                             0.000701   
                     2                             0.001780   

                                  Orbital Period Value  \
radius_bin insol_bin st_Teff_bin                         
4          4         2                        3.836317   
                     2                        3.315605   

                                  Orbital Period Uncertainty  \
radius_bin insol_bin st_Teff_bin                               
4          4         2                              0.000229   
                     2                              0.000480   

                                  Transit Duration (hours) Value  \
radius_bin insol_bin st_Teff_bin                                   
4          4         2                                  2.389717   
                     2                                  1.530639   

                                  Transit Duration (hours) Uncertainty  \
radius_bin insol_bin st_Teff_bin                                         
4          4         2                                        0.084547   
                     2                                        0.340000   

                                  Transit Depth Value  \
radius_bin insol_bin st_Teff_bin                        
4          4         2                    5578.265805   
                     2                    2598.870257   

                                  Transit Depth Uncertainty Sectors  \
radius_bin insol_bin st_Teff_bin                                      
4          4         2                            120.81916       4   
                     2                              5.77319      15   

                                 Public Comment  Surface Gravity Value  \
radius_bin insol_bin st_Teff_bin                                         
4          4         2                 WASP-156                4.60108   
                     2                      NaN                4.54000   

                                  Surface Gravity Uncertainty  Signal ID  \
radius_bin insol_bin st_Teff_bin                                           
4          4         2                                   0.00          1   
                     2                                   0.09          1   

                                  Star Radius Value  Star Radius Uncertainty  \
radius_bin insol_bin st_Teff_bin                                               
4          4         2                     0.760000                     0.00   
                     2                     0.809032                     0.02   

                                  Planet Radius Value  \
radius_bin insol_bin st_Teff_bin                        
4          4         2                       5.635860   
                

In [102]:
binned.loc[:,['TOI',rp_key,mp_key,Ts_key,'Comments','TSM','priority']][binned['TSM'] > 600].sort_values('TSM', ascending=False)

,,,TOI,Planet Radius (R_Earth),pl_masses,Stellar Teff (K),Comments,TSM,priority
radius_bin,insol_bin,st_Teff_bin,,,,,,,
3,3,1,278.01,2.630575,7.434309,2955.0,possible dipper; not a planet candidate; Rstar...,647.611359,1.0
5,3,1,507.01,9.812343,69.689615,3338.0,V-shaped; large planet,634.688014,1.0


In [30]:
tess[tess[id_key] == 1252]

,Source Pipeline,TIC,Full TOI ID,TOI Disposition,TIC Right Ascension,TIC Declination,TMag Value,TMag Uncertainty,Orbital Epoch Value,Orbital Epoch Uncertainty,Orbital Period Value,Orbital Period Uncertainty,Transit Duration (hours) Value,Transit Duration (hours) Uncertainty,Transit Depth Value,Transit Depth Uncertainty,Sectors,Public Comment,Surface Gravity Value,Surface Gravity Uncertainty,Signal ID,Star Radius Value,Star Radius Uncertainty,Planet Radius Value,Planet Radius Uncertainty,Planet Equilibrium Temperature (K) Value,Effective Temperature Value,Effective Temperature Uncertainty,Effective Stellar Flux Value,Signal-to-noise,Centroid Offset,TFOP Master,TFOP SG1a,TFOP SG1b,TFOP SG2,TFOP SG3,TFOP SG4,TFOP SG5,Alerted,Edited,Stellar Mass


In [25]:
#np.savetxt(fname='ticid_names_test.txt',X=tess[['TIC ID']],fmt='%d',header='TICID', delimiter=',')
tess['TIC'].to_csv('ticid_names_only_oct31.csv',header='False',index=False)

In [ ]:
#look for ESM targets as well 
#alison stall wrote code to take candidates and output important diagnostics (from ephemerides)
#make sure data wrangling works so it's adaptable as new TOIs come out

#prioritize NIRSPEC questions bc Emily meeting (find other planets observed)
    
'''
re: NSF - crafting Webb legacy survey might be good. If you say - find TKS sample, that's not so interesting 
more broadly, science question is explore diversity of exoplanet atmospheres. 
Could invision picking off interesting targets w/ found masses and put in observing proposals 
Jonathan: can we identify disequilibrium processes in atmospheres - can learn how to recognize them (even in giant planets)
    and will be better positioned to find them in the future 
    Webb will be sending us data 2022 
    ERS sample (Webb exoplanet sample) might be good
    Study systematics; red noise; etc. in Webb? 
    SHOULD mention Keck observations; SHOULD mention ERS program; SHOULD mention NIRSPEC; SHOULD mention Webb 
        legacy survey
    NEED some kind of science question to answer; we don't necessarily understand what that is yet 
    Search for similar target with Webb
'''

In [398]:
def planck_microns(T,wavelength):
    '''
    note input is in microns! 
    '''
    c = 3e8 #m/s
    h = 6.626e-34
    kb = 1.38e-23
    wav_meters = wavelength / 1e6
    B = 2 * h * c / ((wav_meters**5) * (np.exp(h*c/(wav_meters * kb * T))-1))
    return B

def get_ESM(planet_teq, star_teff, planet_radius, star_radius, Kmag, rp_units = 'jupiter',rs_units='solar'):
    
    dayside_teq = 1.1 * planet_teq
    planet_b = planck_microns(dayside_teq,4)
    star_b = planck_microns(star_teff,4) #note: may want to use 7.5 instead of 4 since the prefactor in Kempton
                                            #is scaled based on the LRS bandpass for a specific planet
                                            #Although I should still get consistent *relative* ESM values. 
    
    if rp_units == 'earth':
        rp_m = 6.371e6 * planet_radius
    elif rp_units == 'jupiter':
        rp_m = 6.9911e7 * planet_radius
    if rs_units == 'solar':
        rs_m = 6.9551e8 * star_radius
    
    ESM = 4.29e6 * (planet_b / star_b) * ((rp_m/rs_m)**2) * (10**(-Kmag/5))
    
    return ESM

NOTE

TKS is using Teff < 6500, NAtalie 10^5, and Rstar < 5.5 
after this, want a file with prioritization 
want to prioritize all P = 1, 2, 3 targets 
       read in confirmed planets from exoplanet archive and compare quality 
       give priority to empty cells 
       give priority to p = 1 (of course) 
       naively, just sort by p; however, how to sort within priority 1? 
       i.e., is there anything better in a "cell" (bin) - how many? 
write out CSV file that could be read into excel. Include a bunch of stuff (P + S properties) 
    for P: R, P, Fp, Mp (if available - create binary value for "was it calculated or measured") 
        name: put name and alternate name (e.g. TOI xxx.xx, WASP-xx) 
        toggles: 0, 1 for "is Kepler, is K2, is TESS") 
    for S: logg, Rs, Vmag, Jmag, Teff, Rs
    also, TSM, ESM, priority, Kamp, numeric value for cell # 
    would be nice if we could look at the # and know immediately where it resides 
    do a three digit integer: want it as Rp, Fp, Teff (low to high) 
    there will be additional historical info and a comments field. Append new comments using | or ; 
        e.g.: "on TKS target list", "previously observed", "vetting status": on confluence, there was a list of criteria that 
           Theo was using related to this 
                could do something fancy, e.g. include links to vetting reports (TESS TOI working group does this when they 
                create spreadsheets; there are cells with links to vetting reports) 
        also want observability: e.g. Mar Apr May it has airmass above 2. (would put "MarAprMay" as "345") - I should think aby 
        this
    something like Theo's visibility plots (airmass as function of UT) - might put this as "mean hours per night as a function 
    of month"
    last thing: what if there are 5 confirmed planets in a cell, and they're all better than the TESS target? Or a new TESS 
    release demotes a formerly P = 1 target? I think a "highest lifetime priority" column might work. Could also do a string 
    which is a concatenation of all previous priorities (e.g. "1111111222222233000000" for something that's steadily getting 
    kicked off)
    Natalie wants to know if a target has been observed with RV measurements. Like, what's the quality of the RV signal (not 
    worry about that yet) 
For Webb: how many planets do we need in a bin? Maybe at least 4 (one for every epoch), maybe 3 (2/3 will be hazy?) maybe 12 (for both of those things)
Also: if we've already observed something for, like, 2 months, why on earth would we get rid of it?? 


In [73]:
kp[kp[id_key]=='GJ 1214 b']

,Planet Radius Value,Stellar Mass,Orbital Period Value,Star Radius Value,Effective Temperature Value,J mag,V mag,TIC Declination,Effective Stellar Flux Value,pl_masses,nTSM,Full TOI ID,TSM,Ars


More from Natalie: 

make sure that we're actually getting the Sector 14, 15, 16 targets (SPOC)